In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go # Plotly graph objects!
import statsmodels.formula.api as sm

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold, TimeSeriesSplit
#from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_text

# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor

# statsmodels: pretty and good to use, great for interpretable ML
from statsmodels.formula.api import ols, logit
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from IPython.display import Image

# !pip install hide_code
# !jupyter nbextension install --py --user hide_code
# !jupyter nbextension enable --py --user hide_code
# !jupyter serverextension enable --py --user hide_code

In [2]:
import os
os.getcwd()
os.chdir('/Users/az/data_projects/ca_pfl/FMLA')

### Import Datasets and Copy

In [3]:
capfl_o = pd.read_csv('ca_edd_data_05_2024.csv')
capfl = capfl_o.copy(deep=True)

income_o = pd.read_csv('med_house_income_ca_2023.csv')
income = income_o.copy(deep=True)

# births_o = pd.read_csv('ca_births_clean_3.csv')
# births = births_o.copy(deep=True)

# labor_o = pd.read_csv('Labor_Force_Participation_Rate__US_and_California_2.csv')
# labor = labor_o.copy(deep=True)

# states_o = pd.read_csv('states_pfl_new.csv')
# states = states_o.copy(deep=True)

# survey = pd.io.stata.read_stata('WHD_FMLA2018EmployeePUF_StataDataSet_Aug2020.dta')
# survey.to_csv('fmla_dataset.csv')

In [4]:
def year_to_month(df, time_index):
    df.set_index('date', inplace=True)
    df = df.resample('M').ffill()
    df = df.reindex(time_index).ffill()
    df = df.reset_index()
    df.rename(columns={'index': 'date'}, inplace=True)
    df.set_index('date', inplace=True)

    return df

### Median Income Data

In [5]:
income.head()

,DATE,MEHOINUSCAA646N
0,2005-01-01,51760
1,2006-01-01,55320
2,2007-01-01,55730
3,2008-01-01,57010
4,2009-01-01,56130


In [6]:
income.tail()

,DATE,MEHOINUSCAA646N
14,2019-01-01,78100
15,2020-01-01,77650
16,2021-01-01,81580
17,2022-01-01,85300
18,2023-01-01,89870


In [7]:
income.rename(columns={
            'MEHOINUSCAA646N': 'median_income',
            'DATE': 'date'}, 
            inplace=True)
income['median_income_logged'] = np.log10(income['median_income'])

income['median_income_weekly'] = income['median_income'] / 52
income['median_income_weekly_logged'] = np.log10(income['median_income_weekly'])

income['median_55%'] = income['median_income_weekly'] * 0.55

income['date'] = pd.to_datetime(income['date'])

income

,date,median_income,median_income_logged,median_income_weekly,median_income_weekly_logged,median_55%
0,2005-01-01,51760,4.713994,995.384615,2.997991,547.461538
1,2006-01-01,55320,4.742882,1063.846154,3.026879,585.115385
2,2007-01-01,55730,4.746089,1071.730769,3.030086,589.451923
3,2008-01-01,57010,4.755951,1096.346154,3.039948,602.990385
4,2009-01-01,56130,4.749195,1079.423077,3.033192,593.682692
5,2010-01-01,54280,4.734640,1043.846154,3.018636,574.115385
6,2011-01-01,53370,4.727297,1026.346154,3.011294,564.490385
7,2012-01-01,57020,4.756027,1096.538462,3.040024,603.096154
8,2013-01-01,60790,4.783832,1169.038462,3.067829,642.971154
9,2014-01-01,60490,4.781684,1163.269231,3.065680,639.798077


In [8]:
t_index = pd.DatetimeIndex(pd.date_range(start='2005-01-01', end='2023-12-31', freq='M'))

income = year_to_month(income, t_index)

In [9]:
income.tail(15)

,median_income,median_income_logged,median_income_weekly,median_income_weekly_logged,median_55%
date,,,,,
2022-10-31,85300.0,4.930949,1640.384615,3.214946,902.211538
2022-11-30,85300.0,4.930949,1640.384615,3.214946,902.211538
2022-12-31,85300.0,4.930949,1640.384615,3.214946,902.211538
2023-01-31,89870.0,4.953615,1728.269231,3.237611,950.548077
2023-02-28,89870.0,4.953615,1728.269231,3.237611,950.548077
2023-03-31,89870.0,4.953615,1728.269231,3.237611,950.548077
2023-04-30,89870.0,4.953615,1728.269231,3.237611,950.548077
2023-05-31,89870.0,4.953615,1728.269231,3.237611,950.548077
2023-06-30,89870.0,4.953615,1728.269231,3.237611,950.548077


### Weekly Benefit Maximum Data

In [10]:
mwba = {
    'year': [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'max_amt': [728, 840, 882, 917, 959, 987, 987, 1011, 1067, 1075, 1104, 1129, 1173, 1216, 1252, 1300, 1357, 1540, 1620],
}

mwba = pd.DataFrame(mwba)

mwba['max_amt_change'] = mwba['max_amt'].pct_change() * 100
mwba['date'] = pd.to_datetime(mwba['year'], format='%Y')

In [11]:
mwba

,year,max_amt,max_amt_change,date
0,2005,728,NaN,2005-01-01
1,2006,840,15.384615,2006-01-01
2,2007,882,5.000000,2007-01-01
3,2008,917,3.968254,2008-01-01
4,2009,959,4.580153,2009-01-01
5,2010,987,2.919708,2010-01-01
6,2011,987,0.000000,2011-01-01
7,2012,1011,2.431611,2012-01-01
8,2013,1067,5.539070,2013-01-01
9,2014,1075,0.749766,2014-01-01


In [12]:
t_index = pd.DatetimeIndex(pd.date_range(start='2005-01-01', end='2023-12-31', freq='M'))

mwba = year_to_month(mwba, t_index)
mwba.tail(15)

,year,max_amt,max_amt_change
date,,,
2022-10-31,2022.0,1540.0,13.485630
2022-11-30,2022.0,1540.0,13.485630
2022-12-31,2022.0,1540.0,13.485630
2023-01-31,2023.0,1620.0,5.194805
2023-02-28,2023.0,1620.0,5.194805
2023-03-31,2023.0,1620.0,5.194805
2023-04-30,2023.0,1620.0,5.194805
2023-05-31,2023.0,1620.0,5.194805
2023-06-30,2023.0,1620.0,5.194805


### Maximum Number of Weeks for Benefit Data

In [13]:
max_weeks = {
    'year': [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'max_num_weeks': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8],
}

max_weeks = pd.DataFrame(max_weeks)
max_weeks['date'] = pd.to_datetime(max_weeks['year'], format='%Y')
max_weeks.tail()

,year,max_num_weeks,date
14,2019,6,2019-01-01
15,2020,8,2020-01-01
16,2021,8,2021-01-01
17,2022,8,2022-01-01
18,2023,8,2023-01-01


In [14]:
t_index = pd.DatetimeIndex(pd.date_range(start='2005-01-01', end='2023-12-31', freq='M'))

max_weeks = year_to_month(max_weeks, t_index)
max_weeks.tail(15)

,year,max_num_weeks
date,,
2022-10-31,2022.0,8.0
2022-11-30,2022.0,8.0
2022-12-31,2022.0,8.0
2023-01-31,2023.0,8.0
2023-02-28,2023.0,8.0
2023-03-31,2023.0,8.0
2023-04-30,2023.0,8.0
2023-05-31,2023.0,8.0
2023-06-30,2023.0,8.0


In [15]:
births.head()

NameError: name 'births' is not defined

In [ ]:
births['date'] = births['year'].astype(str) + '-' + births['month_num'].astype(str)
births['date'] = pd.to_datetime(births['date'],format='%Y-%m')
births.head()

In [ ]:
births = births.set_index('date')
births.index = births.index + pd.offsets.MonthEnd(0)

births.head()

In [ ]:
capfl

In [ ]:
capfl.tail()

In [ ]:
for column in capfl.columns:
    if capfl[column].dtype == 'object':
        capfl[column] = capfl[column].str.replace(",", "")
    elif capfl[column].dtype in ['int64', 'float64']: 
        capfl[column] = capfl[column].astype(str).str.replace(",", "").astype(float)

In [ ]:
capfl = capfl.rename(columns= {'Date': 'old_date'})
capfl['date'] = pd.to_datetime(capfl['old_date'])
capfl.date.dtype

In [ ]:
capfl.dtypes

In [ ]:
new_dtypes_int = {}
int_dtypes = ["Total PFL First Claims Filed", "Bonding Claims Filed", "Care Claims Filed", "Total PFL First Claims Paid", "Bonding Claims Paid","Care Claims Paid", "Weeks Compensated", "Weeks Compensated"]

for item in int_dtypes:
    new_dtypes_int.update({item: "int64"}) 

capfl = capfl.astype(new_dtypes_int)
# capfl.dtypes

In [ ]:
capfl['Total Benefits Authorized'] = capfl['Total Benefits Authorized'].str.replace("$","").astype('float64')

In [ ]:
new_dtypes_obj = {}

for column in capfl.columns:    
    if capfl[column].dtype == 'object':
        new_dtypes_obj.update({column: "category"}) 

capfl = capfl.astype(new_dtypes_obj)
capfl.dtypes

In [ ]:
capfl['Month'] = capfl.Month.cat.reorder_categories(
    new_categories = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                     'October', 'November', 'December'],
    ordered = True)

In [ ]:
capfl.columns

In [ ]:
capfl.head()

In [ ]:
# Logging variables
capfl['Total Benefits Authorized Logged'] = np.log10(capfl['Total Benefits Authorized'])
# capfl['log_bond_file'] = np.log10(capfl['bond_file'])

In [ ]:
# 2004 and 2024 removed from CA PFL dataset (not enough data for 2004 or 2024)
capfl_sub = capfl[~capfl['Year'].isin([2004, 2024])].copy()

In [ ]:
# capfl_clean['missing_claims_tot'] = capfl_clean['tot_pfl_filed'] - capfl_clean['tot_pfl_paid']
# capfl_clean.head()

In [ ]:
capfl_sub.head()

In [ ]:
# Checking for NAs
capfl_sub.isna().sum()

In [ ]:
capfl_sub[capfl_sub['Average Duration'].isna()]

In [ ]:
# Interpolating missing data for average duration
capfl_sub['Average Duration'] = capfl_sub['Average Duration'].interpolate(method = 'linear')

In [ ]:
# Checking that interpolation worked
capfl_sub.isna().sum()

In [ ]:
capfl_sub.set_index(['date'], drop = True, inplace = True)

In [ ]:
capfl_sub = capfl_sub.sort_values('date')

In [ ]:
capfl_sub.head()

In [ ]:
capfl_sub.tail()

In [ ]:
monthly_counts = capfl_sub['Year'].resample('M').count()
monthly_counts.index = monthly_counts.index.to_period('M').to_timestamp()

In [ ]:
monthly_counts.isna().value_counts()

# Merging Datasets

In [ ]:
capfl_merged = pd.merge(capfl_sub, income, 'outer', left_index=True, right_index=True)

In [ ]:
capfl_merged.head()

In [ ]:
capfl_merged = pd.merge(capfl_merged, mwba, 'outer', left_index=True, right_index=True)

In [ ]:
capfl_merged.tail()

In [ ]:
capfl_merged = pd.merge(capfl_merged, max_weeks, 'outer', left_index=True, right_index=True)

In [ ]:
capfl_merged.tail()

In [ ]:
capfl_merged = capfl_merged.drop(columns = ['year_x', 'year_y'])

In [ ]:
capfl_merged.columns

In [ ]:
capfl_merged.to_csv('ca_edd_2023.csv')

In [ ]:
colorblind_palette = ['#1f77b4', '#ff7f0e']

trace1 = go.Scatter(x=pfl_comb_y['year'], 
                    y=pfl_comb_y['mwba_d'], 
                    name = "Max Weekly Benefit",
                   line=dict(color=colorblind_palette[0]))

trace2 = go.Scatter(x=med_inc['year'], 
                    y=med_inc['med_inc_max_wk'], 
                    name="Median Income in CA in Weeks @ 65%", 
                    line=dict(color=colorblind_palette[1]))


fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)


layout = go.Layout(
    title='Median Income in CA in Weeks @ 65% vs. Maximum Weekly Benefit Amount',
    plot_bgcolor='white', 
    xaxis=dict(title='Year', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='$ amount', showgrid=True, gridcolor='lightgray'),
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.show()



In [ ]:
# print('Individuals are taking close to, but not the maximum amount of allowed time off.')
# print('In 2019 leave increased to 8 weeks. In the first 3 years, people took more time, but not the max.')
# print('This shows that claimants take advantage of the benefit.')

colorblind_palette = ['#1f77b4', '#ff7f0e']

trace1 = go.Scatter(x=grouped_avg_duration['year'], y=grouped_avg_duration['avg_duration'], 
                    name = "Avg Time Taken (Weeks)",
                   line=dict(color=colorblind_palette[0]))


trace2 = go.Scatter(x=week_max['year'], y=week_max['week_max_d'], 
                    name = "Max duration",
                   line=dict(color=colorblind_palette[1]))

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)


layout = go.Layout(
    title='Average Time Taken vs. Max Amount of Allowed Time Off',
    plot_bgcolor='white',  # Set the background color to white
    xaxis=dict(title='Year', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='Number of Weeks', showgrid=True, gridcolor='lightgray')
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.show() 

In [ ]:
# print('This tells us that more people use the benefit for bonding claims vs. caregiving claims.')

colorblind_palette = ['#1f77b4', '#ff7f0e']

trace1 = go.Scatter(x=grouped_bond_avg['year'], 
                    y=grouped_bond_avg['bond_claims_tot'], 
                    name = "Bonding Claims",
                   line=dict(color=colorblind_palette[0]))

trace2 = go.Scatter(x=grouped_care_avg['year'], 
                    y=grouped_care_avg['care_claims_tot'], 
                    name = "Caregiver Claims",
                   line=dict(color=colorblind_palette[1]))

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

# fig.update_layout(
#     title="Bonding Claims and Caregiver Claims Over Time",
#     xaxis=dict(title="Year"),
#     yaxis=dict(title="Claims Total"))

layout = go.Layout(
    title='Bonding Claims and Caregiver Claims Over Time',
    plot_bgcolor='white',  # Set the background color to white
    xaxis=dict(title='Year', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='Total Claims', showgrid=True, gridcolor='lightgray')
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.show()  
    


In [ ]:
# print('This shows us that while the birth rate in CA is declining, the number of bonding claims are increasing.')
# print('There should be close to a 2:1 ratio of claims to births thus indicating a lower takeup of the benefit overall from the eligibile population.')

colorblind_palette = ['#1f77b4', '#ff7f0e']

trace1 = go.Scatter(x=grouped_bond_log_avg['year'], y=grouped_bond_log_avg['log_bond_claims_tot'], 
                    name = "Bonding Claims Filed (Logged)",
                   line=dict(color=colorblind_palette[0]))
trace2 = go.Scatter(x=grouped_births_log['year'], y=grouped_births_log['log_num_births'], 
                    name = "Number of Births in CA (Logged)",
                   line=dict(color=colorblind_palette[1]))

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

layout = go.Layout(
    title='Bonding Claims vs. Number of Births in CA',
    plot_bgcolor='white',  # Set the background color to white
    xaxis=dict(title='Year', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='Thousands (Logged)', showgrid=True, gridcolor='lightgray')
)

fig = go.Figure(data=[trace1, trace2], layout=layout)


fig.show()